# Load Resnet Model

In [2]:
import torch

import torch.nn as nn

import torchvision.models as models
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
classes = ['ADM-8',
 'DKRP-4_MinefieldClearance',
 'Dog_Landmine',
 'IGS-50',
 'MOB_Landmine',
 'MON-100',
 'MON-200',
 'MON-50',
 'MON-90',
 'MZM-2',
 'MZU-S',
 'OMZ-4',
 'OZM-160',
 'OZM_Landmine',
 'PFM-1',
 'PMD-6&7',
 'PMK-1_Portable_Mining_Kit',
 'PMN-2',
 'PMN-3',
 'PMN-4',
 'PMN_Landmine',
 'PMP_Landmine',
 'POB-Pilka',
 'POM-1',
 'POM-2',
 'POM-3',
 'POMZ-2',
 'POMZ-2M',
 'PTKM-1R',
 'PTM-1S',
 'PTM-25',
 'PTM-3',
 'PTM-4',
 'PVM_Landmine',
 'SM-320_Signnal_Flare',
 'TM-35',
 'TM-35M',
 'TM-38',
 'TM-39',
 'TM-41',
 'TM-46',
 'TM-56',
 'TM-57',
 'TM-62B',
 'TM-62D',
 'TM-62M',
 'TM-62P',
 'TM-62P2',
 'TM-62P3',
 'TM-62T',
 'TM-71',
 'TM-83',
 'TM-89',
 'TMB-1',
 'TMD-B',
 'TMK-2',
 'Temp-30',
 'UDSh_Landmine',
 'UR-83P_Minefield_Clearance',
 'Yam_Landmine',
 'ZRP-2_Minefield_Clearance',
 'ozm-3',
 'ozm-72',
 'pmk-40',
 'pmm-3',
 'pmm-5',
 'pmz-40']

# Define the model architecture (should match the saved model)
model_pt2 = models.resnet18(pretrained=True)
 
 
lyr4_=model_pt2.layer4[1]

model_pt2.layer4[1].conv2 = nn.Sequential(

    nn.BatchNorm2d(512),

    nn.ReLU(inplace=True),

    nn.Dropout2d(p=0.4)

)
 
 
model_pt2.layer4[1].bn2 = nn.Identity()
 
# The rest of the model remains intact

model_pt2.avgpool = nn.AdaptiveAvgPool2d((1, 1))

model_pt2.fc = nn.Linear(512, len(classes))
 
# Unfreeze all layers

for param in model_pt2.parameters():

    param.requires_grad = True
 
# Move to GPU

model_pt2.to(device)
 
# Modify the last layer to match the number of classes in your dataset

num_classes = 67 # Assuming 'classes' is defined in a previous cell

model_pt2.fc = nn.Linear(model_pt2.fc.in_features, num_classes)
 
# Load the saved state dictionary

checkpoint_path = "best_resnet18.pth" # Use the path where your best model is saved
if not torch.cuda.is_available():
    print("CUDA is not available. Using CPU for inference.")

checkpoint = torch.load(checkpoint_path, map_location=device)

model_pt2.load_state_dict(checkpoint['model_state_dict'])
 
# Move the model to the appropriate device

model_pt2.to(device)
 
# Set the model to evaluation mode

model_pt2.eval()
 
print("Model loaded successfully for inference.")
 
# best_resnet18.pth
 

Model loaded successfully for inference.


In [5]:
# Load image and classify it
from PIL import Image
from torchvision import transforms  
# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the input size of the model
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])
# Load the image
image_path = "pmm-3.jpg"  # Replace with your image path   
image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format
# Apply the transformations
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
# Move the tensor to the appropriate device
image_tensor = image_tensor.to(device)
# Perform inference
with torch.no_grad():
    outputs = model_pt2(image_tensor)
# Get the predicted class index
_, predicted_idx = torch.max(outputs, 1)
predicted_class = classes[predicted_idx.item()]
# Print the predicted class
print(f"Predicted class: {predicted_class}")
# Print the top 5 predicted classes name and indices
_, top5_idx = torch.topk(outputs, 5)    
print("Top 5 predicted classes: ", top5_idx)
# Print the top 5 predicted classes names
top5_classes = [classes[idx.item()] for idx in top5_idx[0]]
print("Top 5 predicted classes names: ", top5_classes)

Predicted class: pmm-3
Top 5 predicted classes:  tensor([[64,  3, 60,  8, 44]], device='cuda:0')
Top 5 predicted classes names:  ['pmm-3', 'IGS-50', 'ZRP-2_Minefield_Clearance', 'MON-90', 'TM-62D']
